# 🕵️‍♂️ ZIPSA Full-Flow & Prompt Debugger (Sandbox Mode)

이 노트북은 서비스의 운영 환경(`src/agents/*.py`)과 동일한 로직으로 **전체 에이전트 그래프의 라우팅 흐름**을 테스트하고 디버깅하기 위해 제작되었습니다.

## 1. 초기화 (Initialization)
프로젝트 환경을 로드하고 **테스트용 샌드박스 프롬프트(`prompts.yaml`)** 환경으로 전환합니다.

In [ ]:
import os
import sys
import asyncio
import shutil
import importlib
import json
from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(project_root)
load_dotenv(os.path.join(project_root, ".env"))

# 🔄 [최종] 모든 관련 모듈 강제 리로드
import src.core.prompts.prompt_manager
import src.agents.state
import src.retrieval.hybrid_search
import src.agents.head_butler
import src.agents.matchmaker
import src.agents.care_team
import src.agents.graph

importlib.reload(src.core.prompts.prompt_manager)
importlib.reload(src.agents.state)
importlib.reload(src.retrieval.hybrid_search) # 리트리버 로그 확인용
importlib.reload(src.agents.head_butler)
importlib.reload(src.agents.matchmaker)
importlib.reload(src.agents.care_team)
importlib.reload(src.agents.graph)

from src.core.prompts.prompt_manager import prompt_manager
from src.agents.state import AgentState
from src.agents.graph import create_zipsa_graph
app = create_zipsa_graph()

print(f"✅ Project Root: {project_root}")
print("✅ LangGraph Application Reloaded (Hard-Routing & Retriever Logs Active).")
print(f"✅ Current Prompt File: {os.path.basename(prompt_manager._yaml_path)}")

## 2. Graph Visualization
`Head Butler`로부터 시작되는 전체 라우팅 구조를 시각화합니다.

In [ ]:
## app의 graph를 시각화
try:
    # get_graph()를 통해 구조를 가져와 머메이드 형태로 렌더링합니다.
    display(Image(app.get_graph().draw_mermaid_png()))
    print("✅ Graph visualization successfully rendered.")
except Exception as e:
    print(f"⚠️ Visualization skipped: {e}")

## 3. Full Flow Trace (Sandbox Execution)
사용자의 질문 입력부터 최종 전문가의 답변까지, 어느 노드를 거쳐가는지 실시간으로 추적합니다.
**테스트용 파일의 프롬프트가 적용된 결과를 확인할 수 있습니다.**

In [ ]:
async def run_debug_trace(query_text: str, user_profile: dict = None):
    if user_profile is None:
        user_profile = {
            "housing": "apartment",
            "activity": "low",
            "traits": ["순한", "조용한"]
        }
    
    print(f"🔍 [USER QUERY]: {query_text}")
    print("=" * 80)
    
    inputs = {
        "route_history": [],
        "messages": [HumanMessage(content=query_text)],
        "user_profile": user_profile
    }
    
    config = {"configurable": {"thread_id": "sandbox_trace_debug_final"}}
    
    # 그래프 실행 및 스트리밍 (updates 모드로 모든 노드 출력)
    async for event in app.astream(inputs, config=config, stream_mode="updates"):
        # 1. 원본 이벤트 딕셔너리 출력
        print(f"{event}")
        
        # 2. 가독성을 위한 추가 상세 정보
        for node_name, values in event.items():
            print(f"📍 [DEBUG]: Node '{node_name}' executed.")
            
            # 메시지 객체 요약 출력
            if "messages" in values:
                for msg in values["messages"]:
                    role = "🤖 AI" if isinstance(msg, AIMessage) else "👤 User"
                    content = msg.content.replace('\n', ' ')[:250]
                    print(f"   ↳ {role}: {content}...")
            
            # 라우팅 정보 요약
            if "router_decision" in values:
                print(f"   ↳ Router Decision: {values['router_decision']}")
            
                
            
            # 디버그 정보 (추론 근거 및 RAG 결과) 요약
            if "debug_info" in values:
                debug = values["debug_info"]
                if "reasoning" in debug:
                    print(f"   💡 Reasoning: {debug['reasoning']}")
                if "care_reasoning" in debug:
                    print(f"   💡 Care Reason: {debug['care_reasoning']}")
                if "retrieved_docs" in debug and debug["retrieved_docs"]:
                    doc_titles = [d.get('title', 'Unknown') for d in debug['retrieved_docs']]
                    print(f"   📚 RAG Retrieved Documents: {doc_titles}")
        
        print("-" * 80)

# ✅ 테스트 케이스
test_query = "서울 강남구에 보호된 고양이가 있어?"

await run_debug_trace(test_query)

## 4. 노드별 개별 테스트 (Isolated Node Test)
운영 중인 코드의 특정 노드만 떼어내어 프롬프트나 로직을 실험합니다.

In [ ]:
from src.agents.head_butler import head_butler_node
from src.agents.matchmaker import matchmaker_node
from src.agents.care_team import care_team_node

async def test_specific_node(node_func, state_dict):
    print(f"🧪 Testing Node: {node_func.__name__}")
    result = await node_func(state_dict)
    print(f"📦 Result Update: {result}")

# 예시: Head Butler의 라우팅만 테스트
mock_state = {
    "route_history": [],
    "messages": [HumanMessage(content="고양이가 설사를 해")],
    "user_profile": {}
}

await test_specific_node(matchmaker_node, mock_state)